In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

2024-02-06 23:39:41.912523: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 23:39:42.024327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [3]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    #x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(inputs)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var')(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = tf.square(z_mean)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = tf.square(z_mean)
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [4]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
vae_enc=make_encoder(57,32,16,3)
vae_dec=make_decoder(57,32,16,3)
vae_40MHZ=VAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Different_VAE_Models/non_normed_new_beta_{}_v4_no_batchnorm_clipped_Kl/'.format(beta),save_format='tf')

Epoch 1/100
18000/18000 [==============================] - 47s 2ms/step - loss: 850132284167.8502 - reco_loss: 850132284167.8228 - kl_loss: 97.0635 - val_loss: 28763.4238 - val_reco_loss: 403.0172 - val_kl_loss: 28360.4062 - lr: 0.0010
Epoch 2/100
18000/18000 [==============================] - 46s 3ms/step - loss: 288.1376 - reco_loss: 225.7661 - kl_loss: 31.0757 - val_loss: 1190.3136 - val_reco_loss: 21.4795 - val_kl_loss: 1168.8341 - lr: 0.0010
Epoch 3/100
18000/18000 [==============================] - 45s 2ms/step - loss: 24.0676 - reco_loss: 21.0410 - kl_loss: 2.4523 - val_loss: 435.5163 - val_reco_loss: 9.8400 - val_kl_loss: 425.6763 - lr: 0.0010
Epoch 4/100
18000/18000 [==============================] - 46s 3ms/step - loss: 19.7117 - reco_loss: 18.0687 - kl_loss: 1.4806 - val_loss: 450.3217 - val_reco_loss: 10.7039 - val_kl_loss: 439.6178 - lr: 0.0010
Epoch 5/100
18000/18000 [==============================] - 44s 2ms/step - loss: 18.0096 - reco_loss: 16.6868 - kl_loss: 1.1289 - v

18000/18000 [==============================] - 45s 2ms/step - loss: 9.1013 - reco_loss: 9.0009 - kl_loss: 0.0995 - val_loss: 35.3801 - val_reco_loss: 8.6437 - val_kl_loss: 26.7364 - lr: 1.0000e-04
Epoch 40/100
18000/18000 [==============================] - 45s 2ms/step - loss: 8.7374 - reco_loss: 8.6404 - kl_loss: 0.0973 - val_loss: 34.1517 - val_reco_loss: 7.6725 - val_kl_loss: 26.4793 - lr: 1.0000e-04
Epoch 41/100
18000/18000 [==============================] - 45s 2ms/step - loss: 8.9326 - reco_loss: 8.8369 - kl_loss: 0.0947 - val_loss: 33.1318 - val_reco_loss: 7.4063 - val_kl_loss: 25.7255 - lr: 1.0000e-04
Epoch 42/100
18000/18000 [==============================] - 43s 2ms/step - loss: 8.7961 - reco_loss: 8.7031 - kl_loss: 0.0928 - val_loss: 32.0275 - val_reco_loss: 8.0955 - val_kl_loss: 23.9320 - lr: 1.0000e-04
Epoch 43/100
18000/18000 [==============================] - 44s 2ms/step - loss: 8.7911 - reco_loss: 8.6997 - kl_loss: 0.0909 - val_loss: 31.9631 - val_reco_loss: 8.0309 - v

In [5]:
a=Data_Train_Flat[0:100]
b=vae_40MHZ.encoder(a)
print(b)

[<tf.Tensor: shape=(100, 3), dtype=float32, numpy=
array([[ 1.00773662e-01,  2.35963464e-02, -4.27330405e-01],
       [-2.83860475e-01,  1.34456813e-01,  4.44695801e-01],
       [-1.10274255e-01,  1.48923874e-01,  1.87409967e-01],
       [-4.33977574e-01, -1.59469247e-02,  2.61006743e-01],
       [ 7.28425980e-02,  8.53597224e-02,  1.83448941e-01],
       [-1.33358508e-01, -2.18769133e-01,  1.24764740e-02],
       [ 4.93590534e-02, -1.99837923e-01, -3.06572318e-02],
       [ 1.68306336e-01,  1.54072315e-01, -5.14794588e-01],
       [ 2.08547428e-01,  3.90993387e-01, -2.09066123e-01],
       [ 5.74311018e-02, -3.43665123e-01, -3.16123396e-01],
       [-2.11883813e-01, -2.01519072e-01,  3.05890143e-02],
       [-9.31852758e-02, -2.25590944e-01,  2.24801004e-02],
       [ 1.28868312e-01,  4.07469153e-01, -2.99613476e-02],
       [-2.23605633e-02,  3.43588471e-01, -2.26695225e-01],
       [-2.03532130e-01,  3.14663380e-01,  4.36624289e-02],
       [ 4.57111001e-01,  2.85661668e-01, -4.8149